# M2 Bending Mode Test

This Jupyter notebook is to run the bending mode test of M2.
Each axial actuator will be issued with an additioal force according to specific bending mode (there are 20).
The target is to understand the mirror supporting system can hold the specific shape based on bending mode or not.

## Import Modules

In [1]:
import asyncio
import yaml
import numpy as np
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK
from lsst.ts.ofc.utils import get_config_dir as get_config_dir_ofc

## Declaration of User-defined Functions

In [2]:
def get_bending_mode():
    """Get the bending mode.

    Returns
    -------
    bending_mode : `numpy.ndarray`
        Bending mode.
    """
    bending_mode_file = get_config_dir_ofc() / "M2" / "M2_1um_72_force.yaml"
    with open(bending_mode_file, "r") as yaml_file:
        bending_mode = np.array(yaml.safe_load(yaml_file))

    return bending_mode

In [3]:
def get_bending_mode_forces(bending_mode, idx_bending_mode, amplitude):
    """Plot the forces of bending mode.
    
    Parameters
    ----------
    bending_mode : `numpy.ndarray`
        Bending mode data.
    idx_bending_mode : `int`
        Index of bending mode (1-20).
    amplitude : `float`
        Amplitude of bending mode in um.
        
    Returns
    -------
    `numpy.ndarray`
        Actuator forces in Newton.
    """

    # Column 0-2 are the actuator ID, x and y position of actuator
    return amplitude * bending_mode[:, 2+idx_bending_mode]

In [4]:
async def apply_force_cycle_axial(csc, forces, time_forces=5, time_reset=5):
    """Apply the force cycle to axial actuators. The cycle will be positive force, clear,
    negative force, clear.

    Parameters
    ----------
    csc : `lsst.ts.salobj.remote.Remote`
        Remote object of the M2 CSC.
    forces : `numpy.ndarray`
        Axial forces to apply (Newton).
    time_forces : `float`, optional
        Time to apply the forces in second. (the default is 5.0) 
    time_reset : `float`, optional
        Time to reset the forces in second. (the default is 5.0)
    """

    # Do the positive direction first
    print(f"Apply the force: {forces} N.")
    await csc.cmd_applyForces.set_start(axial=forces.tolist())
    await asyncio.sleep(time_forces)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(time_reset)

    # Do the Negative direction
    forces_negative = -forces

    print(f"Apply the force: {forces_negative} N.")
    await csc.cmd_applyForces.set_start(axial=forces_negative.tolist())
    await asyncio.sleep(time_forces)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(time_reset)

## Prepare the M2 CSC and put to Enabled state

In [6]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [7]:
state_m2 = m2.evt_summaryState.get()
if state_m2.summaryState != salobj.State.ENABLED:
    await salobj.set_summary_state(m2, salobj.State.ENABLED, timeout=60)

## Apply the Bending Mode

In [8]:
bending_mode = get_bending_mode()

In [12]:
idx_bending_mode = 2
amplitude = 1
forces = get_bending_mode_forces(bending_mode, idx_bending_mode, amplitude)

In [13]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

UTC time to is 2023-05-26 19:31:34.174210 now.


In [14]:
await apply_force_cycle_axial(m2, forces, time_forces=10, time_reset=5)

Apply the force: [-0.1140833   0.1040619   0.3042881   0.4514688   0.5200516   0.4983679
  0.3923987   0.218238    0.00612546 -0.2071172  -0.3842845  -0.49646
 -0.5225269  -0.4575934  -0.3129346  -0.1140832   0.1040615   0.3042879
  0.4514686   0.5200514   0.4983677   0.3923985   0.2182382   0.00612555
 -0.2071169  -0.3842844  -0.49646    -0.5225271  -0.4575937  -0.312935
  0.01614688  0.2033671   0.3356943   0.3777259   0.3190145   0.1753736
 -0.01533061 -0.2020774  -0.335162   -0.3787404  -0.3203629  -0.1756489
  0.01614717  0.2033671   0.3356942   0.3777261   0.3190144   0.1753737
 -0.0153306  -0.2020773  -0.3351619  -0.3787402  -0.3203631  -0.175649
 -0.05672391  0.119974    0.2401183   0.2477969   0.140026   -0.03312396
 -0.1910729  -0.2600001  -0.2069945  -0.05672398  0.119974    0.2401182
  0.2477971   0.1400261  -0.03312382 -0.1910727  -0.2600002  -0.2069943 ] N.
Reset the force.
Apply the force: [ 0.1140833  -0.1040619  -0.3042881  -0.4514688  -0.5200516  -0.4983679
 -0.392398